# Import the necessary Libraries

In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

2022-02-19 13:19:57.805288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-19 13:19:57.805313: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Image Size and Images path for training

In [2]:
# re-size all the images to this
IMAGE_SIZE = [200, 200]

train_path = '/home/vimalkumar/Documents/cool drinks'
folders = glob('/home/vimalkumar/Documents/cool drinks/*')

print(len(folders))

5


# Using VGG16 model for training

In [3]:
vgg16_model = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


2022-02-19 13:20:04.149210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-19 13:20:04.149238: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-19 13:20:04.149252: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-02-19 13:20:04.149424: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Not training the model with existing weights

In [4]:
# don't train existing weights
for layer in vgg16_model.layers:
    layer.trainable = False

# Flattening is done for converting multi-dimensional array into one dimensional array

In [5]:
x = Flatten()(vgg16_model.output)


# Activation function softmax is used because this model contains more than 2 classes i.e (5 classes)

In [6]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16_model.input, outputs=prediction)

In [7]:

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Image Augmentation is done for training Images

In [8]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                  validation_split=0.2)



# Splitting the model for training and testing

In [9]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (200, 200),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                subset = 'training',
                                                 shuffle=True)

test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (200, 200),
                                            batch_size = 16,
                                            class_mode = 'categorical',
                                           subset = 'validation')

Found 3386 images belonging to 5 classes.
Found 844 images belonging to 5 classes.


# Building the VGG16 Model

In [10]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=1,
  #steps_per_epoch=len(training_set),
  #validation_steps=len(test_set)
)

/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/PIL/Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
2022-02-19 13:20:07.426896: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


212/212 [==============================] - 559s 3s/step - loss: 0.2684 - accuracy: 0.9170 - val_loss: 0.1756 - val_accuracy: 0.9431


# Training Accuracy = 91.70%
# Testing Accuracy = 94.31%

# Saving the model

In [11]:
model.save('cooldrinks_model.h5')